In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import pandas as pd
import numpy as np
import os


import gc
import pickle as pickle

from sklearn.model_selection import KFold

from catboost import CatBoostRegressor
import lightgbm as lgbm

import itertools

import multiprocessing as mp
import importlib
from sklearn.model_selection import train_test_split

In [3]:
#tail -n +2 train.csv | split -l 150000
pd.set_option('display.max_columns', None)  
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', -1)

In [4]:
from os import listdir
from os.path import isfile, join

TRAIN_SPLITS='train'
splits = [f for f in listdir(TRAIN_SPLITS) if isfile(join(TRAIN_SPLITS, f))]

In [26]:
TIMESTEPS=1

import build_segment
importlib.reload(build_segment)

from build_segment import build_segment_f

split_chunks = np.array_split(splits,mp.cpu_count())

#def build_segment_f(splits, number_of_groups,test=False, augment=False, scale=True, noise=0.5):

param_test = False
param_augment = False
param_scale = False
param_noise = 0.3
param_smart_augment = False



if __name__ ==  '__main__':
    pool = mp.Pool(mp.cpu_count())
    res = [pool.apply_async(build_segment_f,\
        args=[chunk,TIMESTEPS,param_test,param_augment,param_scale,param_noise,param_smart_augment]) \
           for chunk in split_chunks]
    pool.close()
    pool.join()

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200


In [ ]:
res[0].get()[100]

In [27]:
columns=res[0].get()[0].columns.values

data = pd.DataFrame(columns=columns)


i=0
for r in res:
    for df in r.get():
        data = data.append(df)
        i+=1
        

data.reset_index(drop=True,inplace=True)


In [ ]:
data.sample(10)

In [ ]:
for column in data.columns.values:
    #col_mean = data[column].mean()
    if "augmented" not in column:
        data[f'log_{column}'] =  np.log(data[column])

In [ ]:
combinations2 = list(itertools.combinations(data.columns.values, 2))
#combinations3 = list(itertools.combinations(data.columns.values, 3))


In [ ]:

for combination in combinations2:
    if 'augmented' in combination or 'time_to_failure' in combination:
        continue
    f1 = combination[0]
    f2 = combination[1]
    feature = f'{f1}_mult_{f2}'
    data[feature] = data[f1] * data[f2]
    feature = f'{f1}_plus_{f2}'
    data[feature] = data[f1] + data[f2]
    feature = f'{f1}_div_{f2}'
    #print(feature)
    data[feature] = data[f1] / data[f2]
    data[feature] = pd.to_numeric(data[feature], downcast='float')


In [ ]:

for combination in combinations3:
    if 'augmented' in combination or 'time_to_failure' in combination:
        continue
    f1 = combination[0]
    f2 = combination[1]
    f3 = combination[2]
    feature = f'{f1}_mult_{f2}_mult_{f3}'
    data[feature] = data[f1] * data[f2] * data[f3]
    feature = f'{f1}_plus_{f2}_plus_{f3}'
    data[feature] = data[f1] + data[f2] + data[f3]
    feature = f'{f1}_div_{f2}_div_{f3}'
    #print(feature)
    data[feature] = data[f1] / data[f2] / data[f3]
    data[feature] = pd.to_numeric(data[feature], downcast='float')
    

In [ ]:
splits

shared_train_data, shared_val_data = train_test_split(splits, test_size=0.1, random_state=42)

import pickle

pickle.dump(shared_train_data, open( "shared_train_data.pickle", "wb" ))
pickle.dump(shared_val_data, open( "shared_val_data.pickle", "wb" ))

In [28]:
#drop = ['augmented', 'segment', 'time_to_failure']
drop = ['augmented', 'time_to_failure']


# training__ = data[data['segment'].isin(shared_train_data)]
# train_y = training__['time_to_failure']
# train_data = training__.drop(['segment', 'augmented', 'time_to_failure'], axis=1)

# validation__ = data[data['segment'].isin(shared_val_data)]
# val_y = validation__['time_to_failure']
# val_data = validation__.drop(['segment', 'augmented', 'time_to_failure'], axis=1)

data_ = data.drop(columns=drop)
targets = data['time_to_failure']
test_size = 0.1
train_data, val_data, y_train, y_val = train_test_split(data_, targets, test_size=test_size, random_state=42)

# val_size = int(len(data)*0.1)
# non_augmented = data[data['augmented'] == False]
# val_indices = np.random.choice(non_augmented.index.values, val_size)
# val_data = data[data.index.isin(val_indices)].drop(columns=drop)
# train_data = data[~data.index.isin(val_indices)].drop(columns=drop)

# train_y = train_data['time_to_failure']
# val_y = val_data['time_to_failure']

# train_data = train_data.drop('time_to_failure', axis=1)
# val_data = val_data.drop('time_to_failure', axis=1)

In [ ]:
len(val_data)

In [ ]:
list(all_features)

In [55]:
all_features = train_data.columns.values
cb_features = all_features
lg_features = all_features

In [59]:

cb_model = CatBoostRegressor(iterations=10000,
                             #learning_rate=0.05,
                             eval_metric='MAE',
                             task_type = "GPU",
                             use_best_model=True,
                             od_type = "Iter",
                             od_wait = 50,
                             #depth=4,
                             #l2_leaf_reg=500,
                             #random_strength=500,
                             #border_count=254,
                             #bagging_temperature = 30,
                             #cat_features=[0],
                             random_seed = 42)



cb_model.fit(train_data[cb_features], y_train, #cat_features=categorical_features_indices,
             eval_set=(val_data[cb_features],y_val),
             #cat_features=categorical_features_pos,         
             verbose=True)

cb_scores = {}
for i,score in enumerate(cb_model.get_feature_importance()):
    cb_scores[cb_features[i]] = score

sorted(cb_scores.items(), key=lambda x: x[1])[::-1]

0:	learn: 5.5082772	test: 5.6219355	best: 5.6219355 (0)	total: 7.75ms	remaining: 1m 17s
1:	learn: 5.3536911	test: 5.4731294	best: 5.4731294 (1)	total: 14.6ms	remaining: 1m 13s
2:	learn: 5.2040638	test: 5.3284517	best: 5.3284517 (2)	total: 21.4ms	remaining: 1m 11s
3:	learn: 5.0596257	test: 5.1895578	best: 5.1895578 (3)	total: 27.6ms	remaining: 1m 9s
4:	learn: 4.9213244	test: 5.0587553	best: 5.0587553 (4)	total: 36.1ms	remaining: 1m 12s
5:	learn: 4.7873381	test: 4.9315691	best: 4.9315691 (5)	total: 43.3ms	remaining: 1m 12s
6:	learn: 4.6588174	test: 4.8075018	best: 4.8075018 (6)	total: 50.1ms	remaining: 1m 11s
7:	learn: 4.5359201	test: 4.6909796	best: 4.6909796 (7)	total: 56.8ms	remaining: 1m 11s
8:	learn: 4.4172075	test: 4.5784857	best: 4.5784857 (8)	total: 63.7ms	remaining: 1m 10s
9:	learn: 4.3043203	test: 4.4707040	best: 4.4707040 (9)	total: 70ms	remaining: 1m 9s
10:	learn: 4.1958771	test: 4.3660418	best: 4.3660418 (10)	total: 76.6ms	remaining: 1m 9s
11:	learn: 4.0914923	test: 4.266973

118:	learn: 2.0316662	test: 2.1860780	best: 2.1860780 (118)	total: 802ms	remaining: 1m 6s
119:	learn: 2.0310356	test: 2.1848301	best: 2.1848301 (119)	total: 810ms	remaining: 1m 6s
120:	learn: 2.0302773	test: 2.1838507	best: 2.1838507 (120)	total: 817ms	remaining: 1m 6s
121:	learn: 2.0304084	test: 2.1835394	best: 2.1835394 (121)	total: 824ms	remaining: 1m 6s
122:	learn: 2.0299174	test: 2.1827544	best: 2.1827544 (122)	total: 831ms	remaining: 1m 6s
123:	learn: 2.0292156	test: 2.1812859	best: 2.1812859 (123)	total: 838ms	remaining: 1m 6s
124:	learn: 2.0290614	test: 2.1808730	best: 2.1808730 (124)	total: 845ms	remaining: 1m 6s
125:	learn: 2.0288101	test: 2.1802589	best: 2.1802589 (125)	total: 853ms	remaining: 1m 6s
126:	learn: 2.0283995	test: 2.1795477	best: 2.1795477 (126)	total: 860ms	remaining: 1m 6s
127:	learn: 2.0277095	test: 2.1789141	best: 2.1789141 (127)	total: 867ms	remaining: 1m 6s
128:	learn: 2.0275282	test: 2.1788397	best: 2.1788397 (128)	total: 874ms	remaining: 1m 6s
129:	learn

235:	learn: 1.9889323	test: 2.1607544	best: 2.1607544 (235)	total: 1.58s	remaining: 1m 5s
236:	learn: 1.9886823	test: 2.1605637	best: 2.1605637 (236)	total: 1.59s	remaining: 1m 5s
237:	learn: 1.9883335	test: 2.1608051	best: 2.1605637 (236)	total: 1.6s	remaining: 1m 5s
238:	learn: 1.9880675	test: 2.1605396	best: 2.1605396 (238)	total: 1.61s	remaining: 1m 5s
239:	learn: 1.9875710	test: 2.1605411	best: 2.1605396 (238)	total: 1.61s	remaining: 1m 5s
240:	learn: 1.9870354	test: 2.1600981	best: 2.1600981 (240)	total: 1.62s	remaining: 1m 5s
241:	learn: 1.9868317	test: 2.1599621	best: 2.1599621 (241)	total: 1.63s	remaining: 1m 5s
242:	learn: 1.9865020	test: 2.1600269	best: 2.1599621 (241)	total: 1.63s	remaining: 1m 5s
243:	learn: 1.9862667	test: 2.1601310	best: 2.1599621 (241)	total: 1.64s	remaining: 1m 5s
244:	learn: 1.9856894	test: 2.1597366	best: 2.1597366 (244)	total: 1.65s	remaining: 1m 5s
245:	learn: 1.9856625	test: 2.1597717	best: 2.1597366 (244)	total: 1.65s	remaining: 1m 5s
246:	learn:

354:	learn: 1.9433925	test: 2.1558283	best: 2.1558283 (354)	total: 2.37s	remaining: 1m 4s
355:	learn: 1.9425112	test: 2.1557399	best: 2.1557399 (355)	total: 2.38s	remaining: 1m 4s
356:	learn: 1.9425117	test: 2.1557396	best: 2.1557396 (356)	total: 2.38s	remaining: 1m 4s
357:	learn: 1.9425122	test: 2.1557393	best: 2.1557393 (357)	total: 2.39s	remaining: 1m 4s
358:	learn: 1.9421138	test: 2.1558371	best: 2.1557393 (357)	total: 2.4s	remaining: 1m 4s
359:	learn: 1.9416867	test: 2.1556880	best: 2.1556880 (359)	total: 2.4s	remaining: 1m 4s
360:	learn: 1.9412229	test: 2.1557324	best: 2.1556880 (359)	total: 2.41s	remaining: 1m 4s
361:	learn: 1.9405606	test: 2.1563805	best: 2.1556880 (359)	total: 2.42s	remaining: 1m 4s
362:	learn: 1.9405613	test: 2.1563802	best: 2.1556880 (359)	total: 2.42s	remaining: 1m 4s
363:	learn: 1.9403110	test: 2.1565748	best: 2.1556880 (359)	total: 2.43s	remaining: 1m 4s
364:	learn: 1.9403117	test: 2.1565745	best: 2.1556880 (359)	total: 2.44s	remaining: 1m 4s
365:	learn: 

446:	learn: 1.9130787	test: 2.1522842	best: 2.1522840 (444)	total: 2.97s	remaining: 1m 3s
447:	learn: 1.9130778	test: 2.1522843	best: 2.1522840 (444)	total: 2.97s	remaining: 1m 3s
448:	learn: 1.9127630	test: 2.1523388	best: 2.1522840 (444)	total: 2.98s	remaining: 1m 3s
449:	learn: 1.9123640	test: 2.1524190	best: 2.1522840 (444)	total: 2.99s	remaining: 1m 3s
450:	learn: 1.9116382	test: 2.1527639	best: 2.1522840 (444)	total: 2.99s	remaining: 1m 3s
451:	learn: 1.9116371	test: 2.1527640	best: 2.1522840 (444)	total: 3s	remaining: 1m 3s
452:	learn: 1.9115683	test: 2.1528174	best: 2.1522840 (444)	total: 3s	remaining: 1m 3s
453:	learn: 1.9111473	test: 2.1521964	best: 2.1521964 (453)	total: 3.01s	remaining: 1m 3s
454:	learn: 1.9109105	test: 2.1521689	best: 2.1521689 (454)	total: 3.02s	remaining: 1m 3s
455:	learn: 1.9106967	test: 2.1523347	best: 2.1521689 (454)	total: 3.03s	remaining: 1m 3s
456:	learn: 1.9105141	test: 2.1522778	best: 2.1521689 (454)	total: 3.04s	remaining: 1m 3s
457:	learn: 1.91

552:	learn: 1.8750844	test: 2.1471395	best: 2.1471395 (552)	total: 3.76s	remaining: 1m 4s
553:	learn: 1.8748064	test: 2.1470616	best: 2.1470616 (553)	total: 3.77s	remaining: 1m 4s
554:	learn: 1.8748134	test: 2.1470700	best: 2.1470616 (553)	total: 3.77s	remaining: 1m 4s
555:	learn: 1.8742803	test: 2.1472851	best: 2.1470616 (553)	total: 3.78s	remaining: 1m 4s
556:	learn: 1.8737000	test: 2.1474540	best: 2.1470616 (553)	total: 3.79s	remaining: 1m 4s
557:	learn: 1.8734556	test: 2.1474763	best: 2.1470616 (553)	total: 3.79s	remaining: 1m 4s
558:	learn: 1.8730580	test: 2.1472454	best: 2.1470616 (553)	total: 3.8s	remaining: 1m 4s
559:	learn: 1.8728458	test: 2.1472084	best: 2.1470616 (553)	total: 3.81s	remaining: 1m 4s
560:	learn: 1.8725274	test: 2.1469596	best: 2.1469596 (560)	total: 3.82s	remaining: 1m 4s
561:	learn: 1.8721985	test: 2.1476202	best: 2.1469596 (560)	total: 3.83s	remaining: 1m 4s
562:	learn: 1.8718092	test: 2.1480146	best: 2.1469596 (560)	total: 3.83s	remaining: 1m 4s
563:	learn:

[('roll_diff_1_negs', 17.05086159358581),
 ('roll_diff_1_pos', 13.878931739941239),
 ('roll_diff_1_q75', 6.172671775573615),
 ('roll_diff_1_ssum', 5.864652349477925),
 ('acoustic_data_ssum', 4.393366170935929),
 ('acoustic_data_abs_sum_chg', 4.341624663201486),
 ('acoustic_data_sum_reoccurr_val', 4.196305913287069),
 ('acoustic_data_abs_diff_head_tail', 4.1356320645253515),
 ('acoustic_data_pos', 3.9370313941617017),
 ('roll_diff_1_pos_tail', 3.5889207666173566),
 ('roll_diff_1_neg_head', 3.4572723614306673),
 ('acoustic_data_sum', 3.4507367488197325),
 ('acoustic_data_reocurring_pct', 3.4427850314735347),
 ('roll_diff_1_neg_tail', 3.1554537578896396),
 ('acoustic_data_negs', 3.0858501496383997),
 ('roll_diff_1_skewness', 2.9186783576092123),
 ('roll_diff_1_q25', 2.901179987973718),
 ('acoustic_data_mean', 2.8325616720143176),
 ('acoustic_data_pos_tail', 2.6944017003245513),
 ('acoustic_data_q75', 2.6024547613717286),
 ('acoustic_data_median', 1.0017510531332863),
 ('acoustic_data_q50'

In [58]:
cb_features = [item[0] for item in cb_scores.items() if item[1] > 1]

In [ ]:
cb_model.save_model("cb_model")

In [ ]:
lgtrain = lgbm.Dataset(train_data[lg_features], label=y_train)
lgval = lgbm.Dataset(val_data[lg_features], label=y_val)


# needs to keep overfiting under control, default params don't cut it at all
params = {
        "num_threads": 8,
        "verbosity": -1,
        #"zero_as_missing": "true",
        "boosting":'gbdt',
        "objective" : "regression",
        "metric" : "mae",
        "seed": 42,
        "learning_rate" : 0.05,
        #"min_data_in_leaf": 100,
        "num_leaves": 4,
        #"max_depth" : 4,
        #"bagging_fraction": 0.7,
        #"bagging_freq": 1,
        #"feature_fraction": 0.7,
        #"lambda_l1": 10,
}

evals_result = {}
model_lgb = lgbm.train(params, lgtrain, 10000, 
                      valid_sets=[lgval], 
                      early_stopping_rounds=3, 
                      verbose_eval=10, 
                      evals_result=evals_result)


lg_scores = {}
for i,score in enumerate(model_lgb.feature_importance()):
    lg_scores[lg_features[i]] = score

sorted(lg_scores.items(), key=lambda x: x[1])[::-1]

In [ ]:
lg_features = [item[0] for item in lg_scores.items() if item[1] > 0]

In [ ]:
model_lgb.save_model("lg_model")

In [ ]:
from sklearn.linear_model import ElasticNet, Lasso, Ridge, LinearRegression,BayesianRidge, HuberRegressor

lr_features = cb_features#[0:5]# + lg_features
lr_features = lg_features


lr_model =  LinearRegression()
lr_model.fit(train_data[lr_features], y_train)

from sklearn.metrics import mean_absolute_error

train_preds = lr_model.predict(val_data[lr_features])
print("mae: ", mean_absolute_error(y_val, train_preds))

In [ ]:
rf_features = all_features

In [ ]:
from sklearn.ensemble import RandomForestRegressor

rf_model = RandomForestRegressor(max_depth=5, random_state=0, n_estimators=100)
rf_model.fit(train_data[rf_features], train_y)

val_preds = rf_model.predict(val_data[rf_features])
print("mae: ", mean_absolute_error(val_y, val_preds))

feature_importances = pd.DataFrame(rf_model.feature_importances_,
                        index = rf_features,
                    columns=['importance']).sort_values('importance',ascending=False)
feature_importances

In [ ]:
rf_features = feature_importances[feature_importances['importance'] > 0.005].index.to_list()

In [ ]:
stacked_cb_preds = []
stacked_lg_preds = []
stacked_lr_preds = []
stacked_rf_preds = []


# grouped = data[data['segment'].isin(shared_val_data)].sort_values(by='segment')\
#             .drop(['augmented', 'time_to_failure'], axis=1)\
#             .groupby('segment')


# for name, group in grouped:


# stacked_cb_preds.append(cb_model.predict(val_data[cb_features])[0])
# stacked_lg_preds.append(model_lgb.predict(val_data[lg_features])[0])
# stacked_lr_preds.append(lr_model.predict(val_data[lr_features])[0])
# stacked_rf_preds.append(rf_model.predict(val_data[rf_features])[0])




In [ ]:
stacked_lstm_preds = pickle.load(open("stacked_lstm_val.pickle", "rb"))

In [ ]:
len(stacked_cb_preds)

In [ ]:
#stacked = pd.DataFrame(columns=['cb', 'lg', 'lr', 'rf', 'lstm', 'target'])
stacked = pd.DataFrame(columns=['cb', 'lg', 'lr', 'rf', 'target'])

# stacked['cb'] = stacked_cb_preds
# stacked['lg'] = stacked_lg_preds
# stacked['lr'] = stacked_lr_preds
# stacked['rf'] = stacked_rf_preds

stacked['cb'] = cb_model.predict(val_data[cb_features])
stacked['lg'] = model_lgb.predict(val_data[lg_features])
stacked['lr'] = lr_model.predict(val_data[lr_features])
stacked['rf'] = rf_model.predict(val_data[rf_features])

#stacked['lstm'] = stacked_lstm_preds
# stacked['target'] = data[data['segment'].isin(shared_val_data)]\
#             .sort_values(by='segment')['time_to_failure'].values

stacked['target'] = val_y.values

In [ ]:
stacked.describe()

In [ ]:
stacked.sample(10)
#meta_model.coef_

In [ ]:
data['time_to_failure'].describe()

In [ ]:
data.hist(column='time_to_failure')#.show()

In [ ]:
meta_model =  Lasso(positive=True, alpha=0.1)
meta_model.fit(stacked.drop('target',axis=1)[['cb','lr']], stacked['target'])

from sklearn.metrics import mean_absolute_error

meta_preds = meta_model.predict(stacked.drop('target',axis=1)[['cb','lr']])
print("mae: ", mean_absolute_error(meta_preds, stacked['target']))
meta_model.coef_

In [17]:
TEST_SPLITS='test'
test_splits = [f for f in listdir(TEST_SPLITS) if isfile(join(TEST_SPLITS, f))]

In [18]:
test_split_chunks = np.array_split(test_splits,mp.cpu_count())
    
    
param_test = True
param_augment = False
param_scale = False
param_noise = 0.6
param_smart_augment= False


if __name__ ==  '__main__':
    pool = mp.Pool(mp.cpu_count())
    test_res = [pool.apply_async(build_segment_f,args=[chunk,TIMESTEPS,param_test,param_augment,param_scale,param_noise,param_smart_augment]) for chunk in test_split_chunks]
    pool.close()
    pool.join()

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100


In [19]:
columns=test_res[0].get()[0].columns.values

test_data = pd.DataFrame(columns=columns)


i=0
for r in test_res:
    for df in r.get():
        #print(df)
        test_data = test_data.append(df)
        #print(len(test_data))
        i+=1
        

test_data.reset_index(drop=True,inplace=True)
print(len(test_data))

2624


In [ ]:
combinations2 = list(itertools.combinations(test_data.columns.values, 2))
combinations3 = list(itertools.combinations(test_data.columns.values, 3))


In [ ]:

for combination in combinations2:
    if 'seg_id' in combination or 'time_to_failure' in combination:
        continue
    f1 = combination[0]
    f2 = combination[1]
    feature = f'{f1}_mult_{f2}'
    test_data[feature] = test_data[f1] * test_data[f2]
    feature = f'{f1}_plus_{f2}'
    test_data[feature] = test_data[f1] + test_data[f2]
    feature = f'{f1}_div_{f2}'
    #print(feature)
    test_data[feature] = test_data[f1] / test_data[f2]
    test_data[feature] = pd.to_numeric(test_data[feature], downcast='float')


In [ ]:

for combination in combinations3:
    if 'seg_id' in combination or 'time_to_failure' in combination:
        continue
    f1 = combination[0]
    f2 = combination[1]
    f3 = combination[2]
    feature = f'{f1}_mult_{f2}_mult_{f3}'
    test_data[feature] = test_data[f1] * test_data[f2] * test_data[f3]
    feature = f'{f1}_plus_{f2}_plus_{f3}'
    test_data[feature] = test_data[f1] + test_data[f2] + test_data[f3]
    feature = f'{f1}_div_{f2}_div_{f3}'
    #print(feature)
    test_data[feature] = test_data[f1] / test_data[f2] / test_data[f3]
    test_data[feature] = pd.to_numeric(test_data[feature], downcast='float')
    

In [ ]:
ids = test_data['seg_id'].apply(lambda id: id.split('.')[0])

In [ ]:
cb_preds = cb_model.predict(test_data[cb_features])
lg_preds = model_lgb.predict(test_data[lg_features])
lr_preds = lr_model.predict(test_data[lr_features])
rf_preds = rf_model.predict(test_data[rf_features])

In [ ]:
lr_preds = lr_model.predict(test_data[lr_features])

In [ ]:
preds = (lg_preds * 0.5) + (lr_preds * 0.5)


In [ ]:
cb

In [ ]:

submission = pd.DataFrame(ids)
submission.columns = ['seg_id']
submission['time_to_failure'] = cb_preds

submission.to_csv('cb_preds.csv', index=False)

In [ ]:
submission["time_to_failure"].describe()

In [ ]:
submission.head()

In [ ]:
cb_preds = pd.read_csv('cb_preds.csv')['time_to_failure']
lg_preds = pd.read_csv('lg_preds.csv')['time_to_failure']
lr_preds = pd.read_csv('lr_preds.csv')['time_to_failure']
rf_preds = pd.read_csv('rf_preds.csv')['time_to_failure']
lstm_preds = pd.read_csv('lstm_preds.csv')['time_to_failure']




In [ ]:
np.corrcoef([cb_preds, lg_preds, lr_preds, rf_preds])#, lstm_preds])

In [ ]:
stacked_test = pd.DataFrame(columns=['cb', 'lg', 'lr', 'rf', 'lstm'])
stacked_test['cb'] = cb_preds
stacked_test['lg'] = lg_preds
stacked_test['lr'] = lr_preds
stacked_test['rf'] = rf_preds
stacked_test['lstm'] = lstm_preds


final_preds = meta_model.predict(stacked_test)


In [ ]:
submission = pd.DataFrame(ids)
submission.columns = ['seg_id']
submission['time_to_failure'] = final_preds

submission.to_csv('final_preds.csv', index=False)

submission["time_to_failure"].describe()

In [ ]:
submission["time_to_failure"].describe()

In [ ]:
submission["time_to_failure"].describe()

In [ ]:
from sklearn.model_selection import KFold

stacked_cb = []
stacked_lr = []
stacked_y = []

kf = KFold(n_splits=10)
for train_indices, test_indices in kf.split(data):
    train = data.loc[train_indices]
    test = data.loc[test_indices]
    
    drop = ['augmented', 'segment', 'time_to_failure']
    ttf = 'time_to_failure'
    
    cb_model = CatBoostRegressor(iterations=10000,
                             eval_metric='MAE',
                             use_best_model=True,
                             od_type = "Iter",
                             od_wait = 10,
                             random_seed = 42)

    cb_model.fit(train[cb_features], train[ttf], #cat_features=categorical_features_indices,
             eval_set=(test[cb_features],test[ttf]),
             #cat_features=categorical_features_pos,         
             verbose=False)
    
    stacked_cb.append(cb_model.predict(test[cb_features]))
    
    
    
    lr_model =  LinearRegression()
    lr_model.fit(train[lr_features], train[ttf])

    stacked_lr.append(lr_model.predict(test[lr_features]))
    
    stacked_y.append(test[ttf])
                                       
stacked_cb = [item for sublist in stacked_cb for item in sublist]
stacked_lr = [item for sublist in stacked_lr for item in sublist]
stacked_y = [item for sublist in stacked_y for item in sublist]

In [ ]:
stacked_data = pd.DataFrame(columns=['cb','lr', 'target'])
stacked_data['cb'] = stacked_cb
stacked_data['lr'] = stacked_lr
stacked_data['target'] = stacked_y

In [ ]:
stacked_data.describe()

In [ ]:
meta_model =  Ridge()
meta_model.fit(stacked_data.drop('target',axis=1)[['cb','lr']], stacked_data['target'])

from sklearn.metrics import mean_absolute_error

meta_preds = meta_model.predict(stacked_data.drop('target',axis=1)[['cb','lr']])
print("mae: ", mean_absolute_error(meta_preds, stacked_data['target']))
meta_model.coef_

In [ ]:
cb_preds = cb_model.predict(test_data[cb_features])
lr_preds = lr_model.predict(test_data[lr_features])

In [ ]:
preds_data = pd.DataFrame(columns=['cb','lr'])
preds_data['cb'] = cb_preds
preds_data['lr'] = lr_preds


In [ ]:
preds_data.describe()

In [ ]:
preds = meta_model.predict(preds_data)

In [60]:
ids = test_data['seg_id'].apply(lambda id: id.split('.')[0])

submission = pd.DataFrame(ids)
submission.columns = ['seg_id']
submission['time_to_failure'] = cb_model.predict(test_data[cb_features])

submission.to_csv('cb_preds.csv', index=False)

submission["time_to_failure"].describe()

count    2624.000000
mean     5.854107   
std      2.389847   
min     -0.018402   
25%      3.876023   
50%      5.453855   
75%      7.942374   
max      11.906806  
Name: time_to_failure, dtype: float64

In [ ]:
submission[submission['time_to_failure'] < 0]